Este notebook tem função de limpar os datasets que foram extraídos pela equipe Tera btc-dscbc_jan20 que teve por objetivo analisar alguns impactos do COVID-19 nos teores dos tweets no Brasil.

In [111]:
# importar bibliotecas para isso

# bibliotecas gerais
import pandas as pd
import csv
import tqdm

# bibliotecas para limpar texto
import re
import emoji
import unicodedata
from unicodedata import normalize
import nltk
from ast import literal_eval
import gensim
from gensim.utils import simple_preprocess

#biblioteca sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Função para retirar alfanuméricos e dígitos

def strip_characters(text):
    t = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', text)
    t = re.sub(r'\(|\)|:|,|;|\.|’|”|“|\?|%|>|<', '', t)
    t = re.sub(r'/', ' ', t)
    t = t.replace(r"'",'')
    t = re.sub(r'(@\w+)+', '', t)
    t = t.lower()
    return t

In [3]:
# Função pasra remover caracteres especiais

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [4]:
# Função para retirar emojis

def strip_emoji(text):

    new_text = re.sub(emoji.get_emoji_regexp(), r"", str(text))

    return new_text

In [5]:
# Criando stopwords em português utilizando a biblioteca nltk
import pt_core_news_sm
nlp = pt_core_news_sm.load()
pt_stopwords = sorted([token.text for token in nlp.vocab if token.is_stop])
list_exclude = ['obrigado', 'bom', 'mal', 'nenhuma', 'maior',
             'bem', 'não', 'máximo', 'boa', 'mais',
               'bastante', 'certamente', 'certeza', 'contra',
                'quarentena', 'coronavírus', 'presidente', 'impeachment', 'demitido', 'demitida']
for word in list_exclude:
    nlp.vocab[word].is_stop = False
list_include = set(['o', 'a', 'tá', 'ta', 'ser', 'pro', 'to', 'tô', 'vc', 'você', 'voce', 'pra',
                    'pq', 'é', 'vou', 'que','tão', 'gt', 'de', 'da', 'do', 'em', 'uma', 'lá',
                    'já', 'no', 'para', 'na', 'com', 'um', 'minha', 'se', 'isso', 'por', 'vou',
                    'os', 'isso', 'como', 'mesmo', 'tenho', 'aqui', 'ele', 'ela', 'quem', 'fazer',
                    'eu', 'só', 'ai', 'mais', 'só', 'querer', 'https', 'ter', 'estar', 'ficar',
                    'dos', 'das', 'vcs', 'tem', 'as', 'mas','ao'
                    'tava', 'nao', 'sao', 'ja', 'so', 'nossa',
                    'nosso', 'estao', 'tco', 'me', 'dia', 'te', 'ver', 'sera', 'porra', 'fez', 'ne',
                    'kkk','kkkkkk', 'puta', 'kkkkkkkk', 'hj', 'afff', 'gbr', 'meu', 'cara', 'guri', 'cmg',
                    'ctg', 'agr', 'pqp', 'vdd', 'eh', 'va', 'obg',
                    'corona','virus','coronavirus','covid','covid19','19'
                   'nem', 'numa', 'num', 'nuns', 'ces', 'voces', 'oce', 'oces', 'kkkk', 'vao', 'via',
                    'hj', 'hoje', 'tudo', 'todo', 'toda',
                    'vir', 'bem','ao','sem','ou','vai', 'dizer', 'entao', 'dizer', 'entao',
                    'tao', 'tu', 'mim', 'mano', 'oq', 'pos', 'dm', 'dps',
                    'coronavirusoutbreak', 'coronavirusPandemic', 'dar', 'vairus',
                    'ainda', 'assim']
                  )
for w in list_include:
    nlp.vocab[w].is_stop = True
stop_words = sorted([token.text for token in nlp.vocab if token.is_stop])

In [6]:
def tokenize(text):
    words = nltk.word_tokenize(text)
    return list(set([word for word in words 
                     if len(word) > 1
                     and not (word.isnumeric() and len(word) is not 4)
                     and (not word.isnumeric() or word.isalpha())] )
               )

In [7]:
# import dos tokens

df_tokens = pd.read_csv('tokens_unicos_10k.csv')

In [8]:
# criação de lista de tokens

lista_token = []

for row in range(0, len(df_tokens)):
    lista_token.append(df_tokens.iloc[row,1])
    
lista_token = set(lista_token)

In [9]:
# função para preprocessar palavras

def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts1 = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts2 = [bigram_mod[doc] for doc in texts1]
    texts3 = [trigram_mod[bigram_mod[doc]] for doc in texts2]
    texts4 = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts3]    
    return texts4

# 03/04

In [113]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_03_04.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [114]:
df = df.iloc[1:,:]

In [115]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'Unnamed: 0', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [51]:
# exportar o DF
df.to_csv('0304_preprocessado.csv')

# 02/04

In [52]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_02_04.csv')

In [53]:
df.head()

,Unnamed: 0,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,0,1245863582627033088,2020-04-02 23:59:59,Bolsonaro Ao Vivo https://t.co/8NqTAHyVnA,NaN,pt,NaN,NaN,"Divinópolis, Brasil"
1,1,1245863582618619904,2020-04-02 23:59:59,não sei o q tá acontecendo cmg nessa quarenten...,NaN,pt,NaN,NaN,"Rio de Janeiro, Brasil"
2,2,1245863581830144002,2020-04-02 23:59:59,jro graças a esse fdp do corona eu vou estar i...,NaN,pt,NaN,NaN,caralho
3,3,1245863581666570240,2020-04-02 23:59:59,@veramagalhaes Vc está tranquila pq o Dória já...,NaN,pt,NaN,NaN,NaN
4,4,1245863579619741697,2020-04-02 23:59:59,"Saudade de aglomerações, corona pf já deu",NaN,pt,NaN,NaN,Brasil


In [54]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'Unnamed: 0', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [55]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1245863582627033088,2020-04-02 23:59:59,Bolsonaro Ao Vivo https://t.co/8NqTAHyVnA,bolsonaro ao vivo,bolsonaro ao vivo,bolsonaro ao vivo,"[ao, bolsonaro, vivo]","['vivo', 'bolsonaro']","[vivo, bolsonaro]"
1,1245863582618619904,2020-04-02 23:59:59,não sei o q tá acontecendo cmg nessa quarenten...,não sei o q tá acontecendo cmg nessa quarenten...,nao sei o q ta acontecendo cmg nessa quarenten...,nao sei o q ta acontecendo cmg nessa quarenten...,"[ja, ta, cmg, sei, acontecendo, nada, choro, q...","['cmg', 'nada', 'ja', 've', 'nao']","[nada, ve]"
2,1245863581830144002,2020-04-02 23:59:59,jro graças a esse fdp do corona eu vou estar i...,jro graças a esse fdp do corona eu vou estar i...,jro gracas a esse fdp do corona eu vou estar i...,jro gracas a esse fdp do corona eu vou estar i...,"[fdp, ir, mereco, sem, esse, corona, imenso, t...","['fdp', 'imenso', 'corona', 'tempo', 'aldeia',...","[fdp, imenso, tempo, aldeia, ir]"
3,1245863581666570240,2020-04-02 23:59:59,@veramagalhaes Vc está tranquila pq o Dória já...,vc está tranquila pq o dória já lhe deu 500mi...,vc esta tranquila pq o doria ja lhe deu 500mi...,vc esta tranquila pq o doria ja lhe deu 500mi...,"[politicos, em, 500mil, contrato, ja, de, chan...","['fake', 'jornalismo', 'chantagem', 'doria', '...","[fake, jornalismo, chantagem, doria, pandemia,..."
4,1245863579619741697,2020-04-02 23:59:59,"Saudade de aglomerações, corona pf já deu",saudade de aglomerações corona pf já deu,saudade de aglomeracoes corona pf ja deu,saudade de aglomeracoes corona pf ja deu,"[aglomeracoes, pf, saudade, corona, deu, ja, de]","['pf', 'saudade', 'corona', 'ja']","[pf, saudade]"


In [56]:
# exportar o DF
df.to_csv('0204_preprocessado.csv')

# 01/04

In [57]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_01_04.csv')

In [58]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,1245501195344646145,2020-04-01 23:59:59,em tempos de pandemia.. o mundo um caos.. e so...,NaN,pt,NaN,NaN,"Recife, Pernambuco"
1,1245501195227213825,2020-04-01 23:59:59,@LuisH_fad @DanPerdigaoo @DiiegoDigs foi muit...,NaN,pt,NaN,NaN,NaN
2,1245501193910194177,2020-04-01 23:59:59,Eu só queria que o Cine Bangue colocasse Yoñlu...,NaN,pt,NaN,NaN,"Cabedelo, Paraíba, Brasil."
3,1245501193431982081,2020-04-01 23:59:59,"@CarlosBolsonaro Obrigado, família Bolsonaro p...",NaN,pt,NaN,NaN,"Campo Alegre, Brasil"
4,1245501193230725121,2020-04-01 23:59:59,ler quarto de despejo nessa quarentena tá send...,NaN,pt,NaN,NaN,Brasil


In [59]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [60]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1245501195344646145,2020-04-01 23:59:59,em tempos de pandemia.. o mundo um caos.. e so...,em tempos de pandemia o mundo um caos e so pen...,em tempos de pandemia o mundo um caos e so pen...,em tempos de pandemia o mundo um caos e so pen...,"[ta, em, deve, tempos, povo, fotos, por, de, l...","['lavar', 'pandemia', 'ter', 'povo', 'so', 'ma...","[lavar, pandemia, povo, mao, mundo, luana, pos..."
1,1245501195227213825,2020-04-01 23:59:59,@LuisH_fad @DanPerdigaoo @DiiegoDigs foi muit...,foi muito bom conhecer voces mas dessa qua...,foi muito bom conhecer voces mas dessa qua...,foi muito bom conhecer voces mas dessa qua...,"[mas, bom, voces, muito, conhecer, passo, eu, ...","['bom', 'voces', 'muito', 'conhecer', 'passo',...","[bom, muito, conhecer, passo, adeus]"
2,1245501193910194177,2020-04-01 23:59:59,Eu só queria que o Cine Bangue colocasse Yoñlu...,eu só queria que o cine bangue colocasse yoñlu...,eu so queria que o cine bangue colocasse yonlu...,eu so queria que o cine bangue colocasse yonlu...,"[cine, em, dessa, de, eu, queria, muito, pra, ...","['filme', 'pandemia', 'muito', 'me', 'acabar',...","[filme, pandemia, muito, acabar, chorar, depoi..."
3,1245501193431982081,2020-04-01 23:59:59,"@CarlosBolsonaro Obrigado, família Bolsonaro p...",obrigado família bolsonaro por implantar o so...,obrigado familia bolsonaro por implantar o so...,obrigado familia bolsonaro por implantar o so...,"[hein, socialismo, brasil, implantar, que, bol...","['hein', 'socialismo', 'brasil', 'implantar', ...","[hein, socialismo, brasil, implantar, bolsonar..."
4,1245501193230725121,2020-04-01 23:59:59,ler quarto de despejo nessa quarentena tá send...,ler quarto de despejo nessa quarentena tá send...,ler quarto de despejo nessa quarentena ta send...,ler quarto de despejo nessa quarentena ta send...,"[sendo, na, minha, um, ta, ler, pra, quarto, a...","['ler', 'quarto', 'acordar']","[ler, quarto, acordar]"


In [61]:
# exportar o DF
df.to_csv('0104_preprocessado.csv')

# 31/03

In [62]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_31_3.csv')

In [63]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,1245138807332253696,2020-03-31 23:59:59,@LeilaneNeubarth Gostei da fala do presidente ...,NaN,pt,NaN,NaN,"Belém, Brasil"
1,1245138807151878145,2020-03-31 23:59:59,Início da quarentena // Metade da quarentena h...,NaN,pt,NaN,NaN,NaN
2,1245138806984134656,2020-03-31 23:59:59,eu olho pro bolsonaro e vejo o reflexo do hitl...,NaN,pt,NaN,NaN,derry 🎈
3,1245138806891843585,2020-03-31 23:59:59,@zitosilva Eu sinto um misto: quero que saia /...,NaN,pt,NaN,NaN,"Recife, Brasil"
4,1245138806543732740,2020-03-31 23:59:59,#bbb20 Bolsonaro trocou a boca pelo cú em rede...,NaN,pt,NaN,NaN,NaN


In [64]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [65]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1245138807332253696,2020-03-31 23:59:59,@LeilaneNeubarth Gostei da fala do presidente ...,gostei da fala do presidente \r\nagradeceu a...,gostei da fala do presidente \r\nagradeceu a...,gostei da fala do presidente \r\nagradeceu a...,"[imprensa, gostei, agradeceu, menos, da, do, p...","['menos', 'presidente']","[menos, presidente]"
1,1245138807151878145,2020-03-31 23:59:59,Início da quarentena // Metade da quarentena h...,início da quarentena metade da quarentena,inicio da quarentena metade da quarentena,inicio da quarentena metade da quarentena,"[da, inicio, metade, quarentena]",[],[]
2,1245138806984134656,2020-03-31 23:59:59,eu olho pro bolsonaro e vejo o reflexo do hitl...,eu olho pro bolsonaro e vejo o reflexo do hitl...,eu olho pro bolsonaro e vejo o reflexo do hitl...,eu olho pro bolsonaro e vejo o reflexo do hitl...,"[medo, quero, vejo, odio, hitler, olho, reflex...","['medo', 'odio', 'hitler', 'olho', 'reflexo', ...","[medo, odio, hitler, olho, reflexo, chorar, bo..."
3,1245138806891843585,2020-03-31 23:59:59,@zitosilva Eu sinto um misto: quero que saia /...,eu sinto um misto quero que saia tenho medo...,eu sinto um misto quero que saia tenho medo...,eu sinto um misto quero que saia tenho medo...,"[misto, medo, quero, assume, bem, tenho, de, e...","['misto', 'medo', 'diferente', 'bem', 'mourao'...","[misto, medo, diferente, mourao, bolsonaro, in..."
4,1245138806543732740,2020-03-31 23:59:59,#bbb20 Bolsonaro trocou a boca pelo cú em rede...,#bbb20 bolsonaro trocou a boca pelo cú em rede...,#bbb20 bolsonaro trocou a boca pelo cu em rede...,#bbb20 bolsonaro trocou a boca pelo cu em rede...,"[trocou, rede, nacional, em, bbb20, pelo, bols...","['rede', 'nacional', 'bolsonaro', 'boca', 'cu']","[rede, nacional, bolsonaro, boca, cu]"


In [66]:
# exportar o DF
df.to_csv('3103_preprocessado.csv')

# 29/03

In [67]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_29.csv')

In [68]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,1244414031575887880,2020-03-29 23:59:59,Desespero nos EUA. @realDonaldTrump chegou a o...,NaN,pt,NaN,NaN,"São Paulo, Brazil"
1,1244414030900596736,2020-03-29 23:59:59,@HaddadDebochado ABESTADOHHH?????? O PRESIDENT...,NaN,pt,NaN,NaN,"Macaé, Brasil"
2,1244414030342750208,2020-03-29 23:59:59,"Secretário da Saúde do Ceará, Dr. Cabeto, test...",NaN,pt,NaN,NaN,"Contagem, Brasil"
3,1244414030070140928,2020-03-29 23:59:59,@RASPUTINDECARV1 Nandão curtindo que o bolsona...,NaN,pt,NaN,NaN,NaN
4,1244414029998759938,2020-03-29 23:59:59,"@delbura Queria, o tema vai ser corona vírus",NaN,pt,NaN,NaN,NaN


In [69]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [70]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1244414031575887880,2020-03-29 23:59:59,Desespero nos EUA. @realDonaldTrump chegou a o...,desespero nos eua chegou a obrigar por lei qu...,desespero nos eua chegou a obrigar por lei qu...,desespero nos eua chegou a obrigar por lei qu...,"[serao, lei, panico, amigos, por, de, respirad...","['serao', 'lei', 'panico', 'eua', 'obrigar', '...","[serao, lei, panico, eua, obrigar, fabricar]"
1,1244414030900596736,2020-03-29 23:59:59,@HaddadDebochado ABESTADOHHH?????? O PRESIDENT...,abestadohhh o presidente realmente estava doente,abestadohhh o presidente realmente estava doente,abestadohhh o presidente realmente estava doente,"[abestadohhh, realmente, presidente, doente, e...","['realmente', 'doente', 'presidente']","[realmente, doente, presidente]"
2,1244414030342750208,2020-03-29 23:59:59,"Secretário da Saúde do Ceará, Dr. Cabeto, test...",secretário da saúde do ceará dr cabeto testa p...,secretario da saude do ceara dr cabeto testa p...,secretario da saude do ceara dr cabeto testa p...,"[testa, saude, novo, coronavirus, cabeto, posi...","['saude', 'novo', 'coronavirus', 'cabeto', 'po...","[saude, novo, cabeto, positivo, dr]"
3,1244414030070140928,2020-03-29 23:59:59,@RASPUTINDECARV1 Nandão curtindo que o bolsona...,nandão curtindo que o bolsonarismo está viran...,nandao curtindo que o bolsonarismo esta viran...,nandao curtindo que o bolsonarismo esta viran...,"[minhas, isso, nandotenho, curtindo, falou, na...","['bolsonarismo', 'me', 'besteira']","[bolsonarismo, besteira]"
4,1244414029998759938,2020-03-29 23:59:59,"@delbura Queria, o tema vai ser corona vírus",queria o tema vai ser corona vírus,queria o tema vai ser corona virus,queria o tema vai ser corona virus,"[queria, tema, vai, corona, virus, ser]","['corona', 'virus']",[]


In [71]:
# exportar o DF
df.to_csv('2903_preprocessado.csv')

# 28/03

In [72]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_28csv.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,id,created_at,full_text,country_code,lang,latitude,longitude,location
1,1244414031575887880,2020-03-29 23:59:59,Desespero nos EUA. @realDonaldTrump chegou a o...,NaN,pt,NaN,NaN,"São Paulo, Brazil"
2,1244414030900596736,2020-03-29 23:59:59,@HaddadDebochado ABESTADOHHH?????? O PRESIDENT...,NaN,pt,NaN,NaN,"Macaé, Brasil"
3,1244414030342750208,2020-03-29 23:59:59,"Secretário da Saúde do Ceará, Dr. Cabeto, test...",NaN,pt,NaN,NaN,"Contagem, Brasil"
4,1244414030070140928,2020-03-29 23:59:59,@RASPUTINDECARV1 Nandão curtindo que o bolsona...,NaN,pt,NaN,NaN,NaN


In [74]:
df = df.iloc[1:,:]

In [75]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
1,1244414031575887880,2020-03-29 23:59:59,Desespero nos EUA. @realDonaldTrump chegou a o...,NaN,pt,NaN,NaN,"São Paulo, Brazil"
2,1244414030900596736,2020-03-29 23:59:59,@HaddadDebochado ABESTADOHHH?????? O PRESIDENT...,NaN,pt,NaN,NaN,"Macaé, Brasil"
3,1244414030342750208,2020-03-29 23:59:59,"Secretário da Saúde do Ceará, Dr. Cabeto, test...",NaN,pt,NaN,NaN,"Contagem, Brasil"
4,1244414030070140928,2020-03-29 23:59:59,@RASPUTINDECARV1 Nandão curtindo que o bolsona...,NaN,pt,NaN,NaN,NaN
5,1244414029998759938,2020-03-29 23:59:59,"@delbura Queria, o tema vai ser corona vírus",NaN,pt,NaN,NaN,NaN


In [76]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [77]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
1,1244414031575887880,2020-03-29 23:59:59,Desespero nos EUA. @realDonaldTrump chegou a o...,desespero nos eua chegou a obrigar por lei qu...,desespero nos eua chegou a obrigar por lei qu...,desespero nos eua chegou a obrigar por lei qu...,"[serao, lei, panico, amigos, por, de, respirad...","['serao', 'lei', 'panico', 'eua', 'obrigar', '...","[serao, lei, panico, eua, obrigar, fabricar]"
2,1244414030900596736,2020-03-29 23:59:59,@HaddadDebochado ABESTADOHHH?????? O PRESIDENT...,abestadohhh o presidente realmente estava doente,abestadohhh o presidente realmente estava doente,abestadohhh o presidente realmente estava doente,"[abestadohhh, realmente, presidente, doente, e...","['realmente', 'doente', 'presidente']","[realmente, doente, presidente]"
3,1244414030342750208,2020-03-29 23:59:59,"Secretário da Saúde do Ceará, Dr. Cabeto, test...",secretário da saúde do ceará dr cabeto testa p...,secretario da saude do ceara dr cabeto testa p...,secretario da saude do ceara dr cabeto testa p...,"[testa, saude, novo, coronavirus, cabeto, posi...","['saude', 'novo', 'coronavirus', 'cabeto', 'po...","[saude, novo, cabeto, positivo, dr]"
4,1244414030070140928,2020-03-29 23:59:59,@RASPUTINDECARV1 Nandão curtindo que o bolsona...,nandão curtindo que o bolsonarismo está viran...,nandao curtindo que o bolsonarismo esta viran...,nandao curtindo que o bolsonarismo esta viran...,"[minhas, isso, nandotenho, curtindo, falou, na...","['bolsonarismo', 'me', 'besteira']","[bolsonarismo, besteira]"
5,1244414029998759938,2020-03-29 23:59:59,"@delbura Queria, o tema vai ser corona vírus",queria o tema vai ser corona vírus,queria o tema vai ser corona virus,queria o tema vai ser corona virus,"[queria, tema, vai, corona, virus, ser]","['corona', 'virus']",[]


In [78]:
# exportar o DF
df.to_csv('2803_preprocessado.csv')

# 27/03

In [79]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_27.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,id,created_at,full_text,country_code,lang,latitude,longitude,location
1,1243689256205406208,2020-03-27 23:59:59,É bom mostrar na tv mesmo o rosto e depoimento...,NaN,pt,NaN,NaN,AM
2,1243689256163463169,2020-03-27 23:59:59,"@Eli3nai_ poste, azar é quarentena",NaN,pt,NaN,NaN,"Santa Catarina, Brasil"
3,1243689255534239744,2020-03-27 23:59:59,@ZemaVieira eu indo pro tecno depois da quaren...,NaN,pt,NaN,NaN,NaN
4,1243689255471325185,2020-03-27 23:59:59,quarentena por favor acabe vc está me matando,NaN,pt,NaN,NaN,sorocaos


In [81]:
df = df.iloc[1:,:]

In [82]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
1,1243689256205406208,2020-03-27 23:59:59,É bom mostrar na tv mesmo o rosto e depoimento...,NaN,pt,NaN,NaN,AM
2,1243689256163463169,2020-03-27 23:59:59,"@Eli3nai_ poste, azar é quarentena",NaN,pt,NaN,NaN,"Santa Catarina, Brasil"
3,1243689255534239744,2020-03-27 23:59:59,@ZemaVieira eu indo pro tecno depois da quaren...,NaN,pt,NaN,NaN,NaN
4,1243689255471325185,2020-03-27 23:59:59,quarentena por favor acabe vc está me matando,NaN,pt,NaN,NaN,sorocaos
5,1243689254401826819,2020-03-27 23:59:59,@carlosmaxhado a mãe de todo mundo tá fumando ...,NaN,pt,NaN,NaN,"Campinas, Brasil"


In [83]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [84]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
1,1243689256205406208,2020-03-27 23:59:59,É bom mostrar na tv mesmo o rosto e depoimento...,é bom mostrar na tv mesmo o rosto e depoimento...,e bom mostrar na tv mesmo o rosto e depoimento...,e bom mostrar na tv mesmo o rosto e depoimento...,"[imundo, depoimento, mesmo, de, chamar, na, do...","['chamar', 'bom', 'imundo', 'depoimento', 'tv'...","[chamar, bom, imundo, depoimento, tv, mostrar,..."
2,1243689256163463169,2020-03-27 23:59:59,"@Eli3nai_ poste, azar é quarentena",poste azar é quarentena,poste azar e quarentena,poste azar e quarentena,"[poste, azar, quarentena]",['azar'],[azar]
3,1243689255534239744,2020-03-27 23:59:59,@ZemaVieira eu indo pro tecno depois da quaren...,eu indo pro tecno depois da quarentena,eu indo pro tecno depois da quarentena,eu indo pro tecno depois da quarentena,"[tecno, depois, pro, da, quarentena, eu, indo]",['depois'],[depois]
4,1243689255471325185,2020-03-27 23:59:59,quarentena por favor acabe vc está me matando,quarentena por favor acabe vc está me matando,quarentena por favor acabe vc esta me matando,quarentena por favor acabe vc esta me matando,"[matando, favor, me, acabe, por, quarentena, v...","['favor', 'me']",[favor]
5,1243689254401826819,2020-03-27 23:59:59,@carlosmaxhado a mãe de todo mundo tá fumando ...,a mãe de todo mundo tá fumando maconha na qua...,a mae de todo mundo ta fumando maconha na qua...,a mae de todo mundo ta fumando maconha na qua...,"[na, ta, sonho, mae, todo, fumando, quarentena...","['sonho', 'mae', 'todo', 'mundo', 'maconha']","[sonho, mae, mundo, maconha]"


In [85]:
# exportar o DF
df.to_csv('2703_preprocessado.csv')

# 26/03

In [86]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_26.csv')

In [87]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,1243326867702198275,2020-03-26 23:59:59,@Juuh_sagawa vc ta fazendo oq o bolsonaro quer😠,NaN,pt,NaN,NaN,"Sorocaba, Brasil"
1,1243326867664494592,2020-03-26 23:59:59,até o final dessa quarentena vou assistir goss...,NaN,pt,NaN,NaN,"Guaraciaba do Norte, Brasil"
2,1243326866871681024,2020-03-26 23:59:59,Nessa quarentena eu tô contribuindo pra fuder ...,NaN,pt,NaN,NaN,"Algarve, Portugal."
3,1243326866750046208,2020-03-26 23:59:59,11° dia de quarentena ❤🎶 https://t.co/Rw5vfLRvt2,NaN,pt,NaN,NaN,"Londrina, Brasil"
4,1243326865886138372,2020-03-26 23:59:59,6º dia de quarentena: já comecei a experimenta...,NaN,pt,NaN,NaN,"Dourados, MS - Brasil"


In [88]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude', 'location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [89]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1243326867702198275,2020-03-26 23:59:59,@Juuh_sagawa vc ta fazendo oq o bolsonaro quer😠,vc ta fazendo oq o bolsonaro quer😠,vc ta fazendo oq o bolsonaro quer,vc ta fazendo oq o bolsonaro quer,"[ta, bolsonaro, oq, quer, fazendo, vc]","['bolsonaro', 'oq']",[bolsonaro]
1,1243326867664494592,2020-03-26 23:59:59,até o final dessa quarentena vou assistir goss...,até o final dessa quarentena vou assistir goss...,ate o final dessa quarentena vou assistir goss...,ate o final dessa quarentena vou assistir goss...,"[gossip, final, assistir, umas, dessa, quarent...","['crtz', 'assistir', 'final']","[crtz, assistir, final]"
2,1243326866871681024,2020-03-26 23:59:59,Nessa quarentena eu tô contribuindo pra fuder ...,nessa quarentena eu tô contribuindo pra fuder ...,nessa quarentena eu to contribuindo pra fuder ...,nessa quarentena eu to contribuindo pra fuder ...,"[to, quero, fuder, pessoas, serem, ver, os, eu...","['fuder', 'ver', 'likes', 'cabelo']","[fuder, likes, cabelo]"
3,1243326866750046208,2020-03-26 23:59:59,11° dia de quarentena ❤🎶 https://t.co/Rw5vfLRvt2,11° dia de quarentena ❤🎶,11 dia de quarentena,11 dia de quarentena,"[de, dia, quarentena]",['dia'],[]
4,1243326865886138372,2020-03-26 23:59:59,6º dia de quarentena: já comecei a experimenta...,6º dia de quarentena já comecei a experimentar...,6o dia de quarentena ja comecei a experimentar...,6o dia de quarentena ja comecei a experimentar...,"[6o, comecei, experimentar, receitas, opcoes, ...","['experimentar', 'comida', 'me', 'salsicha', '...","[experimentar, comida, salsicha]"


In [90]:
# exportar o DF
df.to_csv('2603_preprocessado.csv')

# 24/03

In [91]:
# input do dataframe de 01/04

df = pd.read_csv('corona_tweets_24.csv')

In [92]:
df.head()

,id,created_at,full_text,country_code,lang,latitude,longitude,location
0,1242602092688269312,2020-03-24 23:59:59,@rfalcao13 O primeiro passo é começar o proces...,NaN,pt,NaN,NaN,Itu - SP
1,1242602092596023298,2020-03-24 23:59:59,"os cara tão criticando o cara que ""escreveu a ...",NaN,pt,NaN,NaN,NaN
2,1242602092554051584,2020-03-24 23:59:59,"@yassmincarvalh Kkkkkkkk, irmã eu tô até falan...",NaN,pt,NaN,NaN,"São Luís, Brasil"
3,1242602092474372096,2020-03-24 23:59:59,fui ver o pronunciamento e EI BOLSONARO VAI TO...,NaN,pt,NaN,NaN,gg stan
4,1242602092470173699,2020-03-24 23:59:59,O bolsonaro e toda sua família precisam urgent...,NaN,pt,NaN,NaN,Z.O


In [93]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang', 'latitude', 'longitude','location'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [94]:
df.head()

,id,created_at,full_text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1242602092688269312,2020-03-24 23:59:59,@rfalcao13 O primeiro passo é começar o proces...,o primeiro passo é começar o processo para ti...,o primeiro passo e comecar o processo para ti...,o primeiro passo e comecar o processo para ti...,"[mas, processo, apoiam, insano, politicos, emp...","['processo', 'insano', 'politicos', 'empresari...","[processo, insano, politicos, empresarios, pov..."
1,1242602092596023298,2020-03-24 23:59:59,"os cara tão criticando o cara que ""escreveu a ...","os cara tão criticando o cara que ""escreveu a ...","os cara tao criticando o cara que ""escreveu a ...","os cara tao criticando o cara que ""escreveu a ...","[cego, escreveu, muito, os, esse, que, verme, ...","['muito', 'tao', 'verme', 'bolsonaro', 'cego']","[muito, verme, bolsonaro, cego]"
2,1242602092554051584,2020-03-24 23:59:59,"@yassmincarvalh Kkkkkkkk, irmã eu tô até falan...",kkkkkkkk irmã eu tô até falando coisas p @ qu...,kkkkkkkk irma eu to ate falando coisas p @ qu...,kkkkkkkk irma eu to ate falando coisas p @ qu...,"[falando, irma, to, confiando, diria, kkkkkkkk...","['irma', 'so', 'jamais', 'kkkkkkkk']","[irma, jamais]"
3,1242602092474372096,2020-03-24 23:59:59,fui ver o pronunciamento e EI BOLSONARO VAI TO...,fui ver o pronunciamento e ei bolsonaro vai to...,fui ver o pronunciamento e ei bolsonaro vai to...,fui ver o pronunciamento e ei bolsonaro vai to...,"[fui, tomar, ver, vai, pronunciamento, bolsona...","['tomar', 'ver', 'pronunciamento', 'bolsonaro'...","[tomar, pronunciamento, bolsonaro, cu]"
4,1242602092470173699,2020-03-24 23:59:59,O bolsonaro e toda sua família precisam urgent...,o bolsonaro e toda sua família precisam urgent...,o bolsonaro e toda sua familia precisam urgent...,o bolsonaro e toda sua familia precisam urgent...,"[canalhice, um, onde, precisam, sua, senso, va...","['senso', 'pouco', 'ridiculo', 'bolsonaro', 'f...","[senso, pouco, ridiculo, bolsonaro, familia, u..."


In [95]:
# exportar o DF
df.to_csv('2403_preprocessado.csv')

# 19/03

In [96]:
# input do dataframe de 01/04

df = pd.read_csv('tweets_0319_c_bolsonaro.csv')

In [97]:
df.head()

,status_id,created_at,text,country_code,lang
0,1240790152777281536,2020-03-19,@Paivamanu_ kkkkkkkkkkkkkkkk aproveitar a quar...,NaN,pt
1,1240790152613593090,2020-03-19,@MttzinhuC @uiltoo2 negros de verdade tão com ...,NaN,pt
2,1240790152462700549,2020-03-19,@Atanasio_real @l03n27 o loen falando como iss...,NaN,pt
3,1240790152458506244,2020-03-19,a quarentena ta causando sérios problemas ment...,NaN,pt
4,1240790152454311939,2020-03-19,Vou passar a quarentena com o meu amô 😩✌🏻😉🥰,NaN,pt


In [98]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [99]:
df.head()

,status_id,created_at,text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1240790152777281536,2020-03-19,@Paivamanu_ kkkkkkkkkkkkkkkk aproveitar a quar...,kkkkkkkkkkkkkkkk aproveitar a quarentena p re...,kkkkkkkkkkkkkkkk aproveitar a quarentena p re...,kkkkkkkkkkkkkkkk aproveitar a quarentena p re...,"[keru, kkkkkkkkkkkkkkkk, num, ficar, naum, apr...","['rezar', 'aproveitar', 'num', 'ficar']","[rezar, aproveitar]"
1,1240790152613593090,2020-03-19,@MttzinhuC @uiltoo2 negros de verdade tão com ...,negros de verdade tão com bolsonaro mano\n\n...,negros de verdade tao com bolsonaro mano\n\n...,negros de verdade tao com bolsonaro mano\n\n...,"[negros, com, boy, esquerda, bolsonaro, verdad...","['boy', 'bolsonaro', 'verdade', 'branco', 'man...","[boy, bolsonaro, verdade, branco]"
2,1240790152462700549,2020-03-19,@Atanasio_real @l03n27 o loen falando como iss...,o loen falando como isso é coisa de novo-ric...,o loen falando como isso e coisa de novo-ric...,o loen falando como isso e coisa de novo-ric...,"[falando, loen, mosca, na, coisa, novo-rico-de...","['coisa', 'corona']",[coisa]
3,1240790152458506244,2020-03-19,a quarentena ta causando sérios problemas ment...,a quarentena ta causando sérios problemas ment...,a quarentena ta causando serios problemas ment...,a quarentena ta causando serios problemas ment...,"[aos, ta, causando, problemas, serios, brasile...",[],[]
4,1240790152454311939,2020-03-19,Vou passar a quarentena com o meu amô 😩✌🏻😉🥰,vou passar a quarentena com o meu amô 😩✌🏻😉🥰,vou passar a quarentena com o meu amo,vou passar a quarentena com o meu amo,"[meu, passar, com, amo, quarentena, vou]","['meu', 'passar', 'amo']","[passar, amo]"


In [100]:
# exportar o DF
df.to_csv('1903_preprocessado.csv')

# 18/03

In [101]:
# input do dataframe de 01/04

df = pd.read_csv('tweets_0318_c_bolsonaro.csv')

In [102]:
df.head()

,status_id,created_at,text,country_code,lang
0,1240427765431746560,2020-03-18,Terceiro dia de quarentena e nada mudou. já fi...,NaN,pt
1,1240427765326897152,2020-03-18,@seonghwanlol This é an manifestação against o...,NaN,pt
2,1240427765192699907,2020-03-18,"Família, amigos, adoradores, estou bem!! Não e...",BR,pt
3,1240427765062676481,2020-03-18,quarentena n é nenhum desafio pra mim,NaN,pt
4,1240427764664107009,2020-03-18,@radaronline O pior é ver o ministro da saúde ...,NaN,pt


In [103]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [104]:
df.head()

,status_id,created_at,text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1240427765431746560,2020-03-18,Terceiro dia de quarentena e nada mudou. já fi...,terceiro dia de quarentena e nada mudou já fic...,terceiro dia de quarentena e nada mudou ja fic...,terceiro dia de quarentena e nada mudou ja fic...,"[mudou, to, terceiro, fico, nada, em, sem, aco...","['terceiro', 'nada', 'todo', 'ja', 'casa', 'dia']","[terceiro, nada, casa]"
1,1240427765326897152,2020-03-18,@seonghwanlol This é an manifestação against o...,this é an manifestação against o president bo...,this e an manifestacao against o president bo...,this e an manifestacao against o president bo...,"[uma, bolsonaro, day, had, of, this, support, ...","['bolsonaro', 'this']","[bolsonaro, this]"
2,1240427765192699907,2020-03-18,"Família, amigos, adoradores, estou bem!! Não e...",família amigos adoradores estou bem!! não esto...,familia amigos adoradores estou bem!! nao esto...,familia amigos adoradores estou bem!! nao esto...,"[estou, em, contrai, coronavirus, bem, com, ad...","['coronavirus', 'bem', 'familia', 'casa', 'so'...","[familia, casa]"
3,1240427765062676481,2020-03-18,quarentena n é nenhum desafio pra mim,quarentena n é nenhum desafio pra mim,quarentena n e nenhum desafio pra mim,quarentena n e nenhum desafio pra mim,"[nenhum, desafio, mim, pra, quarentena]","['nenhum', 'desafio', 'mim']","[nenhum, desafio]"
4,1240427764664107009,2020-03-18,@radaronline O pior é ver o ministro da saúde ...,o pior é ver o ministro da saúde até então ma...,o pior e ver o ministro da saude ate entao ma...,o pior e ver o ministro da saude ate entao ma...,"[infantil, saude, ver, da, de, pedido, na, ten...","['infantil', 'entao', 'saude', 'ver', 'compete...","[infantil, saude, competente, coletiva, minist..."


In [105]:
# exportar o DF
df.to_csv('1803_preprocessado.csv')

# 17/03

In [106]:
# input do dataframe de 01/04

df = pd.read_csv('tweets_0317_c_bolsonaro.csv')

In [107]:
df.head()

,status_id,created_at,text,country_code,lang
0,1240065377520037888,2020-03-17,O problema de estar falando toda hora da doenç...,NaN,pt
1,1240065376739889153,2020-03-17,"esse bgl de corona vírus tá chatão já, acaband...",NaN,pt
2,1240065376580485120,2020-03-17,Ou lixão o que vc fez até agora?além de perseg...,NaN,pt
3,1240065376458887173,2020-03-17,@carols_lacerda esse corona virus foi longe de...,NaN,pt
4,1240065376068780033,2020-03-17,kkkkkk o cara do corre ta com corona,NaN,pt


In [108]:
# droppando colunas irrelevantes

df = df.drop(['country_code', 'lang'], axis = 1)

# criando listas para incluir os valores
letras = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    letras.append(strip_characters(df.iloc[row,2]))
    
# inclusão dos dados limpos no datafame
df['txt_letras'] = letras

# remove acentuação de todos os tokens
if __name__ == '__main__':
        from doctest import testmod
        testmod()

# criando listas para incluir os valores        
acentos = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    acentos.append(remover_acentos(df.iloc[row,3]))

# inclusão dos dados limpos no datafame
df['txt_acentos'] = acentos

# criando listas para incluir os valores sem emoji
token_emoji_free = []

# loop para garantir que todas as linhas do dataframe sejam analisadas
for row in range(0, len(df)):
    token_emoji_free.append(strip_emoji(df.iloc[row,4]))
        
# inclusão dos dados limpos de emoji no datafame
df['txt_emoji_free'] = token_emoji_free

# transformando tudo em tokens
df['token'] = df['txt_emoji_free'].apply(lambda x: tokenize(x))

# filtranso os tokens para aparecer apenas os 10 mil mais frequentes
df['token2'] = df.token.apply(lambda x: str(list(set(eval(str(x))).intersection(lista_token))))

# transforma os tokens em lista
lista = list(df['token2'])

# Criando Bigram e Trigram

bigram = gensim.models.Phrases(lista, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[lista], threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Processando os tweets
texto = process_words(lista)

# inserindo no df
df['token3'] = texto

In [109]:
df.head()

,status_id,created_at,text,txt_letras,txt_acentos,txt_emoji_free,token,token2,token3
0,1240065377520037888,2020-03-17,O problema de estar falando toda hora da doenç...,o problema de estar falando toda hora da doenç...,o problema de estar falando toda hora da doenc...,o problema de estar falando toda hora da doenc...,"[falando, hora, da, ja, de, problema, com, gen...","['ja', 'hora', 'diferente', 'corona', 'gente',...","[hora, diferente, gente, problema]"
1,1240065376739889153,2020-03-17,"esse bgl de corona vírus tá chatão já, acaband...",esse bgl de corona vírus tá chatão já acabando...,esse bgl de corona virus ta chatao ja acabando...,esse bgl de corona virus ta chatao ja acabando...,"[ja, ta, esse, com, corona, chatao, bgl, meus,...","['ja', 'corona', 'virus']",[]
2,1240065376580485120,2020-03-17,Ou lixão o que vc fez até agora?além de perseg...,ou lixão o que vc fez até agoraalém de persegu...,ou lixao o que vc fez ate agoraalem de persegu...,ou lixao o que vc fez ate agoraalem de persegu...,"[marido, lixo, cambada, ja, de, assim, vao, co...","['vao', 'gordo', 'marido', 'perseguir', 'vagab...","[gordo, marido, perseguir, vagabundo, presiden..."
3,1240065376458887173,2020-03-17,@carols_lacerda esse corona virus foi longe de...,esse corona virus foi longe demais agora,esse corona virus foi longe demais agora,esse corona virus foi longe demais agora,"[demais, longe, esse, agora, corona, virus, foi]","['demais', 'longe', 'agora', 'corona', 'virus']","[demais, longe, agora]"
4,1240065376068780033,2020-03-17,kkkkkk o cara do corre ta com corona,kkkkkk o cara do corre ta com corona,kkkkkk o cara do corre ta com corona,kkkkkk o cara do corre ta com corona,"[ta, kkkkkk, com, corona, do, cara, corre]","['kkkkkk', 'corona']",[]


In [110]:
# exportar o DF
df.to_csv('1703_preprocessado.csv')